In [1]:
import pandas as pd

In [2]:
cnv = pd.read_csv('../data/METABRIC/data_CNA.txt',sep='\t')
cnv.shape

(22544, 2175)

In [3]:
cnv.head(2)

,Hugo_Symbol,Entrez_Gene_Id,MB-0000,MB-0039,MB-0045,MB-0046,MB-0048,MB-0050,MB-0053,MB-0062,...,MB-5467,MB-5546,MB-5585,MB-5625,MB-5648,MB-6020,MB-6213,MB-6230,MB-7148,MB-7188
0,A1BG,1.0,0,0,-1,0,0,0,0,-1,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
1,A1BG-AS1,503538.0,0,0,-1,0,0,0,0,-1,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0


In [4]:
# clean missing more 20
max(cnv.isnull().sum(axis=0).tolist())


3426

In [6]:
max(cnv.isnull().sum(axis=1).tolist())


41

In [23]:
meta_index_cnv = cnv.columns
meta_index_cnv= meta_index_cnv[2:]
tops = ['Hugo_Symbol']
tops.extend( list(meta_index_cnv))
cnv = cnv[tops]

In [24]:
cnv.shape

(22544, 2174)

In [25]:
(cnv.shape[0] /100)* 20

4508.8

In [26]:
for c in cnv.columns:
    missing = cnv[c].isnull().sum()
    if missing >= 4509:
        del cnv[c]
        print(c)

In [27]:
cnv.shape

(22544, 2174)

In [28]:
(2174 / 100 )*20

434.79999999999995

In [29]:
#Find is a feature is missing more than 20%
missing = cnv.isnull().sum(axis=1).tolist()

todel = list()
for idx, m in enumerate(missing):
    if m >= 435:
        todel.append(idx)
todel

[]

In [11]:
# immputation using KNN 20 neigh
from sklearn import neighbors
import numpy as np

In [32]:
# custom version of  https://github.com/bwanglzu/Imputer.py/blob/master/imputer/imputer.py

class Imputer:
    """Imputer class."""
    def __init():
        return

    def _fit(self, X, column, k=10, is_categorical=False):
        """Fit a knn classifier for missing column.
        - Args:
                X(numpy.ndarray): input data
                column(int): column id to be imputed
                k(int): number of nearest neighbors, default 10
                is_categorical(boolean): is continuous or categorical feature
        - Returns:
                clf: trained k nearest neighbour classifier
        """
        clf = None
        if not is_categorical:
            clf = neighbors.KNeighborsRegressor(n_neighbors=k)
        else:
            clf = neighbors.KNeighborsClassifier(n_neighbors=k)
        # use column not null to train the kNN classifier
        missing_idxes = np.where(pd.isnull(X[:, column]))[0]
        if len(missing_idxes) == 0:
            return None
        X_copy = np.delete(X, missing_idxes, 0)
        X_train = np.delete(X_copy, column, 1)
        # if other columns still have missing values fill with mean
        col_mean = None
        if not is_categorical:
            col_mean = np.nanmean(X, 0)
        else:
            col_mean = np.nanmedian(X, 0)
        for col_id in range(0, len(col_mean) - 1):
            col_missing_idxes = np.where(np.isnan(X_train[:, col_id]))[0]
            if len(col_missing_idxes) == 0:
                continue
            else:
                X_train[col_missing_idxes, col_id] = col_mean[col_id]
        y_train = X_copy[:, column]
        # fit classifier
        clf.fit(X_train, y_train)
        return clf

    def _transform(self, X, column, clf, is_categorical):
        """Impute missing values.
        - Args:
                X(numpy.ndarray): input numpy ndarray
                column(int): index of column to be imputed
                clf: pretrained classifier
                is_categorical(boolean): is continuous or categorical feature
        - Returns:
                X(pandas.dataframe): imputed dataframe
        """
        missing_idxes = np.where(np.isnan(X[:, column]))[0]
        X_test = X[missing_idxes, :]
        X_test = np.delete(X_test, column, 1)
        # if other columns still have missing values fill with mean
        col_mean = None
        if not is_categorical:
            col_mean = np.nanmean(X, 0)
        else:
            col_mean = np.nanmedian(X, 0)
        # fill missing values in each column with current col_mean
        for col_id in range(0, len(col_mean) - 1):
            col_missing_idxes = np.where(np.isnan(X_test[:, col_id]))[0]
            # if no missing values for current column
            if len(col_missing_idxes) == 0:
                continue
            else:
                X_test[col_missing_idxes, col_id] = col_mean[col_id]
        # predict missing values
        y_test = clf.predict(X_test)
        X[missing_idxes, column] = y_test
        return X

    def knn(self, X, column, k=10, is_categorical=False):
        """Impute missing value with knn.
        - Args:
                X(pandas.dataframe): dataframe
                column(str): column name to be imputed
                k(int): number of nearest neighbors, default 10
                is_categorical(boolean): is continuous or categorical feature
        - Returns:
                X_imputed(pandas.dataframe): imputed pandas dataframe
        """
        X, column = self._check_X_y(X, column)
        clf = self._fit(X, column, k, is_categorical)
        if clf is None:
            return X
        else:
            X_imputed = self._transform(X, column, clf, is_categorical)
            return X_imputed

    def _check_X_y(self, X, column):
        """Check input, if pandas.dataframe, transform to numpy array.
        - Args:
                X(ndarray/pandas.dataframe): input instances
                column(str/int): column index or column name
        - Returns:
                X(ndarray): input instances
        """
        column_idx = None
        X = X.select_dtypes(include=[np.number])

        if isinstance(X, pd.core.frame.DataFrame):
            if isinstance(column, str):
                # get index of current column
                column_idx = X.columns.get_loc(column)
            else:
                column_idx = column
            X = X.as_matrix()
        else:
            column_idx = column
        return X, column_idx

In [33]:
imputer = Imputer()

In [34]:
icnv = cnv.copy()
#for c in icnv.columns:
#    missing = icnv[c].isnull().sum()
#    if missing > 0:
#        imputed = imputer.knn(icnv, c, k=20 )
#        icnv[c] = imputed[c]
        
#a = imputer.knn(icnv, "MB-5104",k=20)       


In [38]:
a = imputer.knn(icnv, 'MB-5104',k=20, is_categorical=True)

In [39]:
type(a)

numpy.ndarray

In [40]:
a

array([[ 0.,  0., -1., ...,  0.,  0.,  0.],
       [ 0.,  0., -1., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0., -1.,  0.],
       ..., 
       [ 0., -1.,  0., ...,  0.,  0., -1.],
       [ 0.,  0., -2., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.]])

In [42]:
for c in icnv.columns:
    missing = icnv[c].isnull().sum()
    if missing > 0:
#        imputed = imputer.knn(icnv, c, k=20 )
        icnv[c] = imputer.knn(icnv, c, k=20, is_categorical=True )
        
#a = imputer.knn(icnv, "MB-5104",k=20)      

In [43]:
icnv.isnull().sum()

Hugo_Symbol    0
MB-0000        0
MB-0039        0
MB-0045        0
MB-0046        0
MB-0048        0
MB-0050        0
MB-0053        0
MB-0062        0
MB-0064        0
MB-0066        0
MB-0068        0
MB-0079        0
MB-0081        0
MB-0083        0
MB-0093        0
MB-0095        0
MB-0097        0
MB-0100        0
MB-0108        0
MB-0113        0
MB-0120        0
MB-0124        0
MB-0134        0
MB-0138        0
MB-0139        0
MB-0148        0
MB-0149        0
MB-0157        0
MB-0171        0
              ..
MB-5104        0
MB-5106        0
MB-5109        0
MB-5131        0
MB-5136        0
MB-5137        0
MB-5140        0
MB-5158        0
MB-5219        0
MB-5255        0
MB-5289        0
MB-5301        0
MB-5302        0
MB-5325        0
MB-5349        0
MB-5385        0
MB-5387        0
MB-5394        0
MB-5413        0
MB-5460        0
MB-5467        0
MB-5546        0
MB-5585        0
MB-5625        0
MB-5648        0
MB-6020        0
MB-6213        0
MB-6230       

In [45]:
max(icnv.isnull().sum())


0

In [47]:
icnv.to_csv('METABRIC_CNV_implaced.txt',index=False,sep='\t')

In [48]:
def transformation(row):

        if row == -2:
            return 0
        if row == -1:
            return .25
        if row == 0:
            return .5
        if row == 1:
            return .75
        if row == 2:
            return 1
        else:
            print(row)

In [49]:
for c in icnv.columns:
    if c == 'Hugo_Symbol':
        pass
    else:
        icnv[c] = icnv[c].apply(transformation)

In [50]:
icnv.to_csv('METABRIC_CNV_implaced_transformed.txt',index=False,sep='\t')